## (1) Setup
Go through the setup instruction on our [Github Wiki](https://github.com/pro3d-space/PRo3D/wiki/Interactive-Co%E2%80%90Regsitration-Workflow#setup), which concludes with executing the first cell.

In [ ]:
# imports
from typing import Any
import requests
import json
import open3d as o3d
import numpy as np
import subprocess

from pro3d_api import Pro3DClient
import data_io as io
import o3d_uitls as o3du

## (2) Coordinate Alignment
In order to align the coordinate system of SRC to TGT, we need to pick feature points on either surface, that represent the same geologic feature. In our example the angular feature of the cape should work out perfectly. Draw a polyline with **linear projection** on SRC and another on TGT. Both polylines, **must** have the **same number of points**, lie on the **same visual features**, and the points have to be **in the same order**. Minimum number is three, but we recommend to use 5 or 6.

❗ Before executing the next cell, make sure the **annotation in the respective viewer is selected**, otherwise the point retrieval will fail.

In [ ]:
# create api clients for SRC and TGT Pro3D instances
src_client = Pro3DClient(port=4321)
tgt_client = Pro3DClient(port=4322)

# Retrieve selected annotation points from two SRC and TGT Pro3D instances
src_points = src_client.get_selected_annotation_points(verbose=True)
tgt_points = tgt_client.get_selected_annotation_points(verbose=True)

In [ ]:
# Compute the rigid transformation matrix using Open3D
transformation_matrix = o3du.compute_rigid_transformation_from_correspondences(tgt_points, src_points)
print(transformation_matrix)

In [ ]:
# Apply transformation to selected surface
tgt_client.apply_transformation_to_selected_surface(transformation_matrix, verbose=True)


# (3) Fine Registration

## (3.1) Creating a TGT surface cutout

In [ ]:
objText = tgt_client.query_annotation_as_obj(verbose=True)
print(objText)

# Save the transformed object to a file
filename_tgt_cutout ="./cutout.obj"
file = open(filename_tgt_cutout, "w")
file.write(objText)
file.close()

io.open_file_in_meshlab(filename_tgt_cutout)


## (3.2) Employing Iterative Closest Point Optimization

In [ ]:
# ICP fine registration
src_filename = "G:/PRo3D/AI-Mars-3D/Co-Registration/20250325_Nukshak_Sol544/m2020-zcam-delta-above-cape-nukshak-sol-544/source/m26_Delta_W_300k_8k_average.obj"

# Load the SRC and TGT meshes
tgt_mesh = o3d.io.read_triangle_mesh(filename_tgt_cutout)
src_mesh = o3d.io.read_triangle_mesh(src_filename)

# Convert meshes to point clouds for registration
tgt_pcd = tgt_mesh.sample_points_uniformly(number_of_points=100000)
src_pcd = src_mesh.sample_points_uniformly(number_of_points=100000)

# Define the pre-alignment (Helmert transformation)
pre_alignment = transformation_matrix

# Apply the pre-alignment transformation to the SRC point cloud
# src_pcd.transform(pre_alignment)

# Perform fine registration using ICP
icp_result = o3d.pipelines.registration.registration_icp(
    src_pcd, tgt_pcd, max_correspondence_distance=0.01,
    init=pre_alignment, # np.eye(4),
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200)
)

# Print the transformation matrices
print("Pre-Alignment (Helmert) Transformation:")
print(pre_alignment)

print("\nICP Refinement Transformation:")
print(icp_result.transformation)

In [ ]:
# Apply final result
tgt_client.apply_transformation_to_selected_surface(icp_result.transformation, verbose=True)

In [ ]:
# Apply transformation to selected surface
tgt_client.apply_transformation_to_selected_surface(transformation_matrix, verbose=True)
